In [1]:
from tool import *
from tqdm import tqdm

In [2]:
# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',60)
pd.set_option('display.max_colwidth', None)

## 1. 양품 S/N 자료를 전수검사 DB에 업데이트

In [113]:
df1 = pd.read_excel(get_filename(), skiprows=2, sheet_name='SN') # QA 전수검사 엑셀파일의 양품S/N 탭 자료 불러옴, 상단에서 2줄 Skip함
df1.columns = df1.columns.astype(str) # 컬럼명을 String으로 변경

#  상기 데이타프레임의 컬럼명에서 날짜만 추출한 것으로 새로운 컬럼명 생성
new_col = [i for i in df1.columns if i.startswith('Unn') != True]
new_col = [i for i in new_col if i.startswith('날') != True]

df1 = df1[new_col] # 새로운 컬럼명으로 데이타프레임 변경

new_col = [] # 새로운 컬럼명에서 날짜명을 잘라서 컬럼명 재변경
for col in df1.columns:
    new_col.append(col[:10])
df1.columns = new_col

# 비어있는 데이타프레임 생성 후 거기에 위에서 만든 데이타프레임의 정보를 추출하여 가공하여 새로운 포맷으로 만듬
df_new = pd.DataFrame()
for num, col in enumerate(df1.columns):
    for sn in df1.iloc[:,num].dropna()[2:]: # 3번째 S/N 데이타부터 가져오기 위해 slicing
        temp_dic = {'Inspection Date':col, 'Mapping Model.Suffix':df1.iloc[:, num][0], 'S/N':sn}
        df_new = pd.concat([df_new, pd.DataFrame.from_dict(temp_dic, orient='index').T])
df_new['Judge'] = 'OK'
df_new['Inspection Date'] = pd.to_datetime(df_new['Inspection Date'])

FileNotFoundError: [Errno 2] No such file or directory: ''

### 전수검사 사유 및 책임 소재 기입

In [4]:
df_new

,Inspection Date,Mapping Model.Suffix,S/N,Judge
0,2023-06-09,15U50R-G.AR3CK,305QCDG562698,OK
0,2023-06-09,15U50R-G.AR3CK,305QCBD562685,OK
0,2023-06-09,15U50R-G.AR3CK,305QCQX562812,OK
0,2023-06-09,15U50R-G.AR3CK,305QCHE562737,OK
0,2023-06-09,15U50R-G.AR3CK,305QCPY562760,OK
...,...,...,...,...
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562901,OK
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562864,OK
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562912,OK
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562973,OK


In [5]:
# 전수검사 사유의 예
reason1 = 'Full inspection for quality control'
reason2 = 'Full inspection for sampling inspection NG( 16T90R : Caps Lock Key Crack, C-part stain, Seal Label peel off)'
reason3 = 'Full inspection for sampling inspection NG(16T90Q LCM noise)'
reason4 = 'Full inspection for sampling inspection NG(Noise problem caused by internal screw influx in the product)'
reason5 = 'Full inspection for sampling inspection NG(15U50R D-part screw missing, System Fan noise)'
reason6 = 'Full inspection for sampling inspection NG(LCD open/close noise issue)'

reason = reason6

df_new['Responsibility'] = 'Supplier'
df_new['Inspection_Reason'] = reason

In [6]:
df_new

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Responsibility,Inspection_Reason
0,2023-06-09,15U50R-G.AR3CK,305QCDG562698,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCBD562685,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCQX562812,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCHE562737,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCPY562760,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
...,...,...,...,...,...,...
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562901,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562864,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562912,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562973,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)


In [7]:
df_new['S/N'].str.len().value_counts() # S/N 자리수가 13자리 맞는지 확인

13    304
Name: S/N, dtype: int64

In [8]:
df_new[~df_new['Mapping Model.Suffix'].str.contains(odm_model)] # OS모델명 형태와 맞지 않는 모델명 있는지 확인

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Responsibility,Inspection_Reason


In [9]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df = pickle.load(f)

In [10]:
df.shape

(111904, 12)

In [11]:
sample = pd.DataFrame() # DB에 update되는 신규 데이타만 취합하기 위해 만든 빈 dataframe
for row in tqdm(df_new.iterrows()):
    cond = (df['Inspection Date'] == row[1]['Inspection Date']) & (df['S/N'] == row[1]['S/N']) & (df['Judge'] == row[1]['Judge']) & (df['Mapping Model.Suffix'] == row[1]['Mapping Model.Suffix'])
    if df[cond].shape[0] == 0:
        df = pd.concat([df, pd.DataFrame(row[1]).T])
        sample = pd.concat([sample, pd.DataFrame(row[1]).T])
df.reset_index(drop=True, inplace=True)

304it [00:27, 11.19it/s]


In [12]:
sample.shape # 신규로 추가된 S/N 개수 확인

(304, 6)

In [13]:
sample

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Responsibility,Inspection_Reason
0,2023-06-09,15U50R-G.AR3CK,305QCDG562698,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCBD562685,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCQX562812,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCHE562737,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15U50R-G.AR3CK,305QCPY562760,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
...,...,...,...,...,...,...
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562901,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562864,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCLH562912,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
0,2023-06-09,15UD40R-G.AX3EK,305QCBD562973,OK,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)


In [14]:
df.shape

(112208, 12)

In [15]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [16]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

## 2. 불량 탭에서 데이타 가져와서 불량 내역을 영어로 번역하기

In [17]:
df1 = pd.read_excel(get_filename(), skiprows=2, sheet_name='불량현황') # 특정 불량 모델의 탭의 자료 가져오기

In [18]:
df1 = df1.loc[:, df1.columns[df1.columns.str.contains('검사')][0]:]
df1.columns = ['Inspection Date', 'Mapping Model.Suffix', 'MFG DATE', 'S/N', 'Defect Reason_Korean', ''] # 컬럼명 변경

In [19]:
df1 = df1.dropna(how='all') # 전체 행에 빈값이 있는것은 다 날림

In [20]:
df1 = df1.drop('', axis=1) # 불필요한 컬럼 삭제

In [21]:
df1.insert(4, 'Judge', 'NG') # judge 컬럼 추가하여 NG로 표시

In [22]:
df1['Responsibility'] = 'Supplier'
df1['Inspection_Reason'] = reason

In [23]:
df1 = df1[df1['S/N'].notnull()]

In [24]:
df1['Defect Reason_Korean'] = df1['Defect Reason_Korean'].str.replace('\n', ',')

In [25]:
df1 # 취합 데이터 확인

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason
0,2023-06-09,15UD40R-G.AX3EK,2023-05-22,33968305QCCV562859,NG,LCD open /close Noise,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
1,2023-06-09,15UD40R-G.AX3EK,2023-05-22,33968305QCWC562945,NG,LCD open /close Noise,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
2,2023-06-09,15UD40R-G.AX3EK,2023-05-22,33968305QCPY563000,NG,LCD open /close Noise,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)
3,2023-06-09,15UD40R-G.AX3EK,2023-05-22,33968305QCVU563047,NG,LCD open /close Noise,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue)


### 1) 번역 (by Google cloud API)

In [26]:
df1['Defect Reason_English'] = df1['Defect Reason_Korean'].apply(get_google_translate, args=['en']) # Google cloud API 월 번역용량 제한있음

In [27]:
df1['Defect Reason_English'] = df1['Defect Reason_English'].str.replace('&quot;', '"')
df1['Defect Reason_English'] = df1['Defect Reason_English'].str.replace('&#39;', '"')

In [28]:
df1[['Defect Reason_Korean', 'Defect Reason_English']][:] # 번역 결과 확인

,Defect Reason_Korean,Defect Reason_English
0,LCD open /close Noise,LCD open /close Noise
1,LCD open /close Noise,LCD open /close Noise
2,LCD open /close Noise,LCD open /close Noise
3,LCD open /close Noise,LCD open /close Noise


### 번역 수정할 때

In [46]:
keyword = 'Key 무감'

In [47]:
df1.loc[df1['Defect Reason_Korean'].str.contains(keyword), 'Defect Reason_English']

1    No key, (no key including F2 key)
Name: Defect Reason_English, dtype: object

In [48]:
df1.loc[df1['Defect Reason_Korean'].str.contains(keyword), 'Defect Reason_English'] = 'Many Key insensitivity(Including F2 Key)'

In [49]:
df1

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason,Defect Reason_English
0,2023-02-28,16TD90R-G.AX76K,2023-02-24,302QCUK564022,NG,LCD 흑점,LG,Full inspection for quality control,LCD black dot
1,2023-02-28,16TD90R-G.AX56K,2023-02-21,302QCTB561977,NG,"Key 무감,(F2 Key 포함 다수 Key 무감)",LG,Full inspection for quality control,Many Key insensitivity(Including F2 Key)
2,2023-02-28,16TD90R-G.AX56K,2023-02-17,302QCUK561934,NG,"D-Part 누름시 소음,(꿀렁꿀렁됨)",LG,Full inspection for quality control,Noise when pressing D-Part


### 2) 번역 (by 구글 번역 Site using 셀레늄)

In [57]:
# 셀레늄 이용하므로 속도 느리나, 한개씩 번역할 때 번역 정확도 높은 편
df1['Defect Reason_English'] = df1['Defect Reason_Korean'].apply(google_translate)

In [58]:
df1 # 번역결과 확인

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Defect Reason_English
0,2022-03-03,17U70P-P.AA7SK,2022-01-26,201PGPM654448,NG,Image 설치 실패 (Error 창 발생),Image installation failure (Error window occurs)
1,2022-03-03,17U70P-P.AA7SK,2022-01-26,201PGBL654397,NG,Image 설치 실패 (Error 창 발생),Image installation failure (Error window occurs)


### 3) 번역 (by 구글 번역 Site using 셀레늄 with large string data)

In [151]:
# 번역해야할 양이 많을 때, 구글 번역하기 위한 불량내용(한국어)를 1개의 text로 만듬.
total_text = ''
for text in df1['Defect Reason_Korean']:
    text = text.replace('\n', ' ')
    total_text = total_text + text + '\n'

In [152]:
len(total_text.split('\n')[:-1]) # 전체 번역할 text의 한줄씩 전체 개수

102

In [153]:
result = google_translate(total_text) # 셀레늄으로 구글 번역한 결과 저장

In [154]:
len(result.split('\n')) # 결과 text 를 한줄씩 전체 개수 확인(위의 개수와 맞는지 확인)

102

In [155]:
df.loc[:, 'Defect Reason_English'] = result.split('\n') # 맞으면, 새로운 불량내용(영어) 컬럼에 번역내용 붙여넣기

In [31]:
df # 결과 확인

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
...,...,...,...,...,...,...,...,...,...,...,...
1854,2022-04-13,14TD90P-G.AXFBK,203QCKJ581469,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1855,2022-04-13,14TD90P-G.AXFBK,203QCKJ581493,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1856,2022-04-13,14TD90P-G.AXFBK,203QCEA581756,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1857,2022-04-13,14TD90P-G.AXFBK,203QCFT581475,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df1.to_excel('trans.xlsx', index=False) # 엑셀로 만들어 번역 재검증

In [159]:
df1 = pd.read_excel(get_filename()) # 재검증한 결과를 다시 df로 만듬

In [32]:
df.columns

Index(['Inspection Date', 'Mapping Model.Suffix', 'S/N', 'Judge', 'MFG DATE',
       'Defect Reason_Korean', 'Defect Reason_English', 'Repair Report',
       'RMA Number', 'Responsibility', 'Inspection_Reason'],
      dtype='object')

In [59]:
df1

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason,Defect Reason_English
0,2022-08-23,14T90Q-G.AR5PK,2022-08-09,208QCGW560431,NG,C-Part Rubber 누락,Supplier,품질안정화를 위한 전수 검사,Missing C-Part Rubber
1,2022-08-23,14T90Q-G.AR5PK,2022-08-09,208QCAS560536,NG,Touch Pad 좌측 누름 시 상하 동시 눌림,Supplier,품질안정화를 위한 전수 검사,Upper and lower part simultaneously pressed wh...


In [4]:
df1 = pd.read_excel(get_filename(), sheet_name='NG')
df1

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Defect Reason_Korean,Defect Reason_English,Responsibility,Inspection_Reason
0,2023-03-30,15UG50Q-G.AR56KN,303QCQX569420,NG,Box damage,Box damage,LG,Box Wet damage Issue
1,2023-03-30,15UG50Q-G.AR56KN,303QCQX569228,NG,Box damage,Box damage,LG,Box Wet damage Issue
2,2023-03-30,15UG50Q-G.AR56KN,303QCRN569534,NG,Box damage,Box damage,LG,Box Wet damage Issue
3,2023-03-30,15UG50Q-G.AR56KN,303QCSF569398,NG,Box damage,Box damage,LG,Box Wet damage Issue
4,2023-03-30,15UG50Q-G.AR56KN,303QCTB569233,NG,Box damage,Box damage,LG,Box Wet damage Issue
...,...,...,...,...,...,...,...,...
1058,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569346,OK,NaN,NaN,LG,Box Wet damage Issue
1059,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569370,OK,NaN,NaN,LG,Box Wet damage Issue
1060,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569418,OK,NaN,NaN,LG,Box Wet damage Issue
1061,2023-03-30,15UG50Q-G.AR56KN,303QCXM569602,OK,NaN,NaN,LG,Box Wet damage Issue


## 3. 불량탭에서 가져온 데이타를 기존 DB에 추가함

In [29]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df = pickle.load(f)

In [30]:
df.shape

(112208, 12)

In [31]:
df1.shape

(4, 9)

In [32]:
sample = pd.DataFrame() # DB에 update되는 신규 데이타만 취합하기 위해 만든 빈 dataframe
for row in tqdm(df1.iterrows()):
    cond = (df['Inspection Date'] == row[1]['Inspection Date']) & (df['S/N'] == row[1]['S/N']) & (df['Judge'] == row[1]['Judge']) & (df['Mapping Model.Suffix'] == row[1]['Mapping Model.Suffix'])
    if df[cond].shape[0] == 0:
        df = df.append(row[1])
        sample = sample.append(row[1])
df.reset_index(drop=True, inplace=True)

0it [00:00, ?it/s]C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20636\3586433766.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row[1])
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20636\3586433766.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(row[1])
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20636\3586433766.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row[1])
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20636\3586433766.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(row[1])
2it [00:00, 11.84it/s]C:\Users\paul76.lee\AppD

In [33]:
sample.shape

(4, 9)

In [34]:
df.shape

(112212, 12)

In [35]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [36]:
df

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112207,2023-06-09,15UD40R-G.AX3EK,305QCDG562914,OK,NaT,NaN,NaN,NaN,NaN,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue),NaN
112208,2023-06-09,15UD40R-G.AX3EK,33968305QCCV562859,NG,2023-05-22,LCD open /close Noise,LCD open /close Noise,NaN,NaN,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue),NaN
112209,2023-06-09,15UD40R-G.AX3EK,33968305QCWC562945,NG,2023-05-22,LCD open /close Noise,LCD open /close Noise,NaN,NaN,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue),NaN
112210,2023-06-09,15UD40R-G.AX3EK,33968305QCPY563000,NG,2023-05-22,LCD open /close Noise,LCD open /close Noise,NaN,NaN,Supplier,Full inspection for sampling inspection NG(LCD open/close noise issue),NaN


In [37]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [38]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

## 4. DOA 수리후 환입되는 정보를 DB에 추가

### 1) 업체별 DOA Return 현황 대비 입고 및 검사 수량 현황

In [3]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

df1 = df1.fillna('TBA').pivot_table('S/N', index=['Vendor', 'DOA Number', 'OBD'], aggfunc='count').reset_index()
df1 = df1.rename(columns={'S/N':'DOA QTY'})

with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

df2 = df2[df2['Import Biz Type']=='NCV'].pivot_table('Shipment Quantity', index=['DOA Number'], aggfunc=sum).reset_index()
df2= df2.rename(columns={'Inspection':'DOA Number', 'Shipment Quantity':'Return QTY'})
df2

df3 = df1.merge(df2, how='left')
df3 = df3.fillna(0)
df3.sort_values(['Vendor', 'OBD'])

with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df4 = pickle.load(f)
df4 = df4[df4['DOA Number'].notnull()]

df4 = df4.pivot_table('S/N', index='DOA Number', aggfunc='count').reset_index()
df4 = df4.rename(columns={'S/N':'Inspection QTY'})

df5 = df3.merge(df4, how='left')
df5 = df5.fillna(0)
df5['Balance QTY'] = df5['DOA QTY'] - df5['Inspection QTY']
df5 = df5.convert_dtypes()
df5

,Vendor,DOA Number,OBD,DOA QTY,Return QTY,Inspection QTY,Balance QTY
0,Pegatron,LG20230410,2023-04-15,14,14,14,0
1,Pegatron,LG20230705,2023-07-08,3,0,0,3
2,Pegatron,LG211006,2021-10-12,35,35,35,0
3,Pegatron,LG211102,2021-11-09,26,26,26,0
4,Pegatron,LG211201,2021-12-09,63,63,63,0
5,Pegatron,LG220304,2022-03-17,51,51,51,0
6,Pegatron,LG220510,2022-05-25,27,27,27,0
7,Pegatron,LG220721,2022-08-01,39,39,39,0
8,Pegatron,LG220818,2022-09-01,562,562,552,10
9,Pegatron,LG221017,2022-10-23,42,42,42,0


### 2) 특정 DOA Number에 대한 모델별 입고/검사 수량 현황

In [4]:
doa_num = ['N5V23625H-391L']

In [5]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

df1 = df1[df1['DOA Number'].isin(doa_num)]
df1 = df1.pivot_table('S/N', index=['Mapping Model.Suffix'], aggfunc='count')
df1 = df1.rename(columns={'S/N':'DOA QTY'})

with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

c1 = (df2['Import Biz Type']=='NCV')
c2 = (df2['DOA Number'].isin(doa_num))
if df2[c1 & c2].shape[0] != 0:
    df2 = df2[c1 & c2].pivot_table('Shipment Quantity', index='Item No', aggfunc=sum)
    df2 = df2.rename(columns={'Shipment Quantity':'Return QTY'})
else:
    df2 = pd.DataFrame(0, index=df1.index, columns=['Return QTY'])
    
with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df3 = pickle.load(f)

if df3[df3['DOA Number'].isin(doa_num)].shape[0] != 0:
    df3 = df3[df3['DOA Number'].isin(doa_num)]
    df3 = df3.pivot_table('S/N', index='Mapping Model.Suffix', aggfunc='count')
    df3 = df3.rename(columns={'S/N':'Inspected QTY'})
else:
    df3 = pd.DataFrame(0, index=df1.index, columns=['Inspected QTY'])

df3 = pd.concat([df1, df2, df3], axis=1)
df3 = df3.convert_dtypes()
df3 = df3.fillna(0)
df3['Balance QTY'] = df3['DOA QTY'] - df3['Inspected QTY']
df3.loc['Sum', :] = df3.sum()
df3

,DOA QTY,Return QTY,Inspected QTY,Balance QTY
Mapping Model.Suffix,,,,
14T90Q-G.AA56K,1,0,0,1
14T90Q-G.AA79K,1,0,0,1
14T90R-G.AA50K,2,0,0,2
14T90R-G.AA5CK,3,0,0,3
14TD90R-G.AX56K,2,0,0,2
15U40Q-G.AR30K,1,0,0,1
15U40R-G.AR30K,5,0,0,5
15U40R-G.ARFWK,2,0,0,2
15U50P-G.AR50ML,1,0,0,1


### 3) DOA Number 중 아직 전수검사 하지 않은 S/N 리스트

In [6]:
def get_balance_doa_unit(doa_num):
    with open('D:/Data/PC ODM DOA return DB.bin', 'rb') as f:
        df1 = pickle.load(f)
    df1 = df1[df1['DOA Number'] == doa_num]
    
    with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
        df2 = pickle.load(f)
    df2 = df2[df2['DOA Number'] == doa_num]
    df3 = df1[~df1['S/N'].isin(df2['S/N'])]
    return df3.reset_index(drop=True)

df = pd.DataFrame()
for lot in doa_num:
    df = pd.concat([df, get_balance_doa_unit(lot)])
df

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark
0,Quanta,14T90Q,14T90Q-G.AA56K,760.1,211QCDG560122,System Fan Noise,System Fan Noise,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
1,Quanta,14T90Q,14T90Q-G.AA79K,872.1,210QCAS557192,A/B-Part 누름 시 소음,Noise when pressing A/B-Part,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
2,Quanta,14T90R,14T90R-G.AA50K,723.38,302QCSF566966,D-Part 누름시 소음,Noise when pressing D-Part,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
3,Quanta,14T90R,14T90R-G.AA50K,723.38,96462303QCAS559040,Mic 동작 불량,Mic malfunction,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
4,Quanta,14T90R,14T90R-G.AA5CK,730.66,08363305QCEA556724,LCD 내부 이물,Foreign matter inside the LCD,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Quanta,16T90Q,16TD90Q-G.AX70K,857.65,204QCDG564194,D-Part 누름시 소음,Noise when pressing D-Part,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
63,Quanta,16T90R,16TD90R-G.AX56K,773.43,302QCTB561977,D-Part 누름시 소음,Noise when pressing D-Part,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
64,Quanta,16T90Q,16TG90Q-G.AA75KN,891.85,302QCQX571116,"KBD(: / "") Key Slant","KBD(: / "") Key Slant",F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN
65,Quanta,16T90Q,16TG90Q-G.AA75KN,891.85,90571304QCTB568089,D-Part Scratch,D-Part Scratch,F1000210782,CPCZ2307050003,N5V23625H-391L,KR101133931,2023-07-08,NaN


In [7]:
df.to_clipboard()

#### 실제 업체로부터 받은 DOA 발송 내역 불러옴

In [11]:
df2 = pd.read_excel(get_filename(), sheet_name=1)
df2 = df2.rename(columns={'Quanta Action':'Repair Report'})
# df2 = df2.rename(columns={'repair action':'Repair Report'})
df2

,Vendor,Series,Mapping Model.Suffix,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),Unnamed: 6,Repair Report
0,Quanta,14T90R,14T90R-G.AA56K,96451303QCHE570977,F Key 들뜸,F key lift,NaN,Replace KB
1,Quanta,14T90R,14T90R-G.AA56K,96451303QCLH571080,IMAGE Install Fail,IMAGE Install Fail,NaN,Repair MB
2,Quanta,14T90R,14T90R-G.AP56ML,24264304QCBD566277,Caps Lock Key Slant,Caps Lock Key Slant,NaN,Replace KB
3,Quanta,14T90R,14T90R-G.AP56ML,24264304QCPY566256,LCD 휘점,LCD bright point,NaN,Replace LCD
4,Quanta,14T90R,14T90R-G.AP56ML,24264304QCXM566226,T/P 벗겨짐,T/P peeling off,NaN,Replace T/P
5,Quanta,15U40R,15U40R-G.ARTWK,97487303QCYQ570786,LCD Open/Close Noise,LCD Open/Close Noise,NaN,Replace B cover
6,Quanta,15U40R,15U40R-G.ARTWK,97487303QCZP570597,동영상 재생 이어폰 연결시 사운드 끊김,Sound cuts out when connecting earphones for video playback,NaN,Repair MB
7,Quanta,15U50R,15U50R-S.AP70ML,24290304QCQX567292,D-Part Screw 누락,Missing D-Part Screw,NaN,Replace D cover
8,Quanta,15U50R,15U50R-S.AP70ML,24290304QCWC567305,D-Part Rubber 파손,Damage to D-Part Rubber,NaN,Replace D cover
9,Quanta,16T90R,16T90R-G.AA5CK,08361304QCGW556111,"Adapter 포장 불량, Box 불량","Bad adapter packaging, bad box",NaN,Repair adapter


In [12]:
df3 = df[df['S/N'].isin(df2['S/N'])]
# df3 = df[df['S/N'].apply(lambda x:x[5:] if len(x)==18 else x).isin(df2['S/N'])]# 람다함수로 해당 스트링의 자리수가 18자리 일때만 5번째 위치부터 slicing 하여 돌려준 것으로 해당 S/N이 환입된 S/N에 있는 것만 추출

In [13]:
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark
0,Quanta,14T90R,14T90R-G.AA56K,723.29,96451303QCHE570977,F Key 들뜸,F key lift,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
1,Quanta,14T90R,14T90R-G.AA56K,723.29,96451303QCLH571080,IMAGE Install Fail,IMAGE Install Fail,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
2,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCBD566277,Caps Lock Key Slant,Caps Lock Key Slant,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
3,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCPY566256,LCD 휘점,LCD bright point,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
4,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCXM566226,T/P 벗겨짐,T/P peeling off,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
5,Quanta,15U40R,15U40R-G.ARTWK,356.77,97487303QCYQ570786,LCD Open/Close Noise,LCD Open/Close Noise,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
6,Quanta,15U40R,15U40R-G.ARTWK,356.77,97487303QCZP570597,동영상 재생 이어폰 연결시 사운드 끊김,Sound cuts out when connecting earphones for video playback,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
7,Quanta,15U50R,15U50R-S.AP70ML,617.26,24290304QCQX567292,D-Part Screw 누락,Missing D-Part Screw,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
8,Quanta,15U50R,15U50R-S.AP70ML,617.26,24290304QCWC567305,D-Part Rubber 파손,Damage to D-Part Rubber,F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN
9,Quanta,16T90R,16T90R-G.AA5CK,798.64,08361304QCGW556111,"Adapter 포장 불량, Box 불량","Bad adapter packaging, bad box",F1000209363,CPCZ2305240003,N5V23522H-391L,KR101132984,2023-05-30,NaN


In [14]:
df3['Repair Report'] = pd.merge(df3['S/N'], df2[['S/N', 'Repair Report']]).drop(columns='S/N')
# df3['Repair Report'] = pd.merge(df3['S/N'].apply(lambda x:x[5:] if len(x)==18 else x), df2[['S/N', 'Repair Report']]).drop(columns='S/N')

In [15]:
df3 = df3.reset_index(drop=True)

In [16]:
inv = df3.groupby(['Mapping Model.Suffix', 'Price'])['S/N'].count()
inv.name = 'QTY'
inv = inv.reset_index()
inv['Amt'] = inv['Price'] * inv['QTY']
inv.loc[len(inv)] = ['Total', '', inv['QTY'].sum(), inv['Amt'].sum()]
inv

,Mapping Model.Suffix,Price,QTY,Amt
0,14T90R-G.AA56K,723.29,2,1446.58
1,14T90R-G.AP56ML,753.19,3,2259.57
2,15U40R-G.ARTWK,356.77,2,713.54
3,15U50R-S.AP70ML,617.26,2,1234.52
4,16T90R-G.AA5CK,798.64,10,7986.40
5,16T90R-G.AA76K,846.73,2,1693.46
6,16T90R-G.AP7WL,876.81,4,3507.24
7,Total,,25,18841.31


In [17]:
str_date = datetime.date.today().strftime('%Y%m%d')
vendor = df3['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find)[0]
doa_qty = df3['S/N'].count()
with pd.ExcelWriter(f'D:/Python result/{str_date}_{vendor}_DOA return list_{doa_qty}.xlsx') as writer:
    df3.drop(columns=['FOC Number', 'Grouping Number', 'HBL', 'OBD', 'Remark']).to_excel(writer, sheet_name='RAW DATA', index=False)
    inv.to_excel(writer, sheet_name='Inv_data', index=False)

## 무환 수입 통관 요청 메일 양식

수신자 : 김준현 사원, 김윤정 선임, 권혁호 주임,  이경훈 기정,  
참조자 : 임달호 기정, 변성권 사원, 이재성 기정, , 백호영 책임, 조희진 선임, lg@esein.co.kr, 이구희 책임  

제목 : Quanta 불량 수리 후 환입 건 무환 통관 요청(BL : xxxx )  

안녕하십니까?

첨부는 중국 PC ODM 업체인 ___ 에서 불량 수리 후 무상 재환입의 건으로 SD 입력하였으니 무환 통관 요청드립니다.
통관 진행시, 관세 0%로 부가세 감면 없이 일반 수입 진행하시기 바랍니다.
제품 수령 담당자는 권혁호 주임이십니다.

이경훈 조장님,

__ 에서 불량 수리 후 재환입의 건이오니 통관 후, 첨부의 수리 내역 참고하시어 입고 검사 부탁 드립니다.

1. 수입제품내역

2. SD 입력 완료 화면


### 4) DOA 환입품 전수검사 내역의 S/N과 일치하는 지 확인

In [6]:
# DOA 환입 리스트를 불러옴
df3 = pd.read_excel(get_filename())
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),DOA Number,Repair Report
0,Quanta,14T90R,14T90R-G.AA56K,723.29,96451303QCHE570977,F Key 들뜸,F key lift,N5V23522H-391L,Replace KB
1,Quanta,14T90R,14T90R-G.AA56K,723.29,96451303QCLH571080,IMAGE Install Fail,IMAGE Install Fail,N5V23522H-391L,Repair MB
2,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCBD566277,Caps Lock Key Slant,Caps Lock Key Slant,N5V23522H-391L,Replace KB
3,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCPY566256,LCD 휘점,LCD bright point,N5V23522H-391L,Replace LCD
4,Quanta,14T90R,14T90R-G.AP56ML,753.19,24264304QCXM566226,T/P 벗겨짐,T/P peeling off,N5V23522H-391L,Replace T/P
5,Quanta,15U40R,15U40R-G.ARTWK,356.77,97487303QCYQ570786,LCD Open/Close Noise,LCD Open/Close Noise,N5V23522H-391L,Replace B cover
6,Quanta,15U40R,15U40R-G.ARTWK,356.77,97487303QCZP570597,동영상 재생 이어폰 연결시 사운드 끊김,Sound cuts out when connecting earphones for video playback,N5V23522H-391L,Repair MB
7,Quanta,15U50R,15U50R-S.AP70ML,617.26,24290304QCQX567292,D-Part Screw 누락,Missing D-Part Screw,N5V23522H-391L,Replace D cover
8,Quanta,15U50R,15U50R-S.AP70ML,617.26,24290304QCWC567305,D-Part Rubber 파손,Damage to D-Part Rubber,N5V23522H-391L,Replace D cover
9,Quanta,16T90R,16T90R-G.AA5CK,798.64,08361304QCGW556111,"Adapter 포장 불량, Box 불량","Bad adapter packaging, bad box",N5V23522H-391L,Repair adapter


In [7]:
# DOA 전수검사 결과 파일 불러옴
df5 = pd.read_excel(get_filename(), skiprows=2, sheet_name='RMA')
df5 = df5.loc[:, '모델명':'불량 내용']
df5

,모델명,S/N,생산지,차용일,불량 증상,수리 내용,판정,불량 내용
0,14T90R-G.AA56K,96451303QCHE570977,Quanta,2023-06-21,F Key 들뜸,Replace KB,OK,NaN
1,14T90R-G.AA56K,96451303QCLH571080,Quanta,2023-06-21,IMAGE Install Fail,Repair MB,OK,NaN
2,14T90R-G.AP56ML,24264304QCBD566277,Quanta,2023-06-21,Caps Lock Key Slant,Replace KB,OK,NaN
3,14T90R-G.AP56ML,24264304QCPY566256,Quanta,2023-06-21,LCD 휘점,Replace LCD,OK,NaN
4,14T90R-G.AP56ML,24264304QCXM566226,Quanta,2023-06-21,T/P 벗겨짐,Replace T/P,OK,NaN
5,15U40R-G.ARTWK,97487303QCYQ570786,Quanta,2023-06-21,LCD Open/Close Noise,Replace B cover,OK,NaN
6,15U40R-G.ARTWK,97487303QCZP570597,Quanta,2023-06-21,동영상 재생 이어폰 연결시 사운드 끊김,Repair MB,OK,NaN
7,15U50R-S.AP70ML,24290304QCQX567292,Quanta,2023-06-21,D-Part Screw 누락,Replace D cover,OK,NaN
8,15U50R-S.AP70ML,24290304QCWC567305,Quanta,2023-06-21,D-Part Rubber 파손,Replace D cover,OK,NaN
9,16T90R-G.AA5CK,08361304QCGW556111,Quanta,2023-06-21,"Adapter 포장 불량, Box 불량",Repair adapter,OK,NaN


In [8]:
# 불러온 파일의 S/N 리스트와 업체가 보낸 S/N 리스트와 일치하는지 확인
set(df5['S/N']) - set(df3['S/N'])

set()

### 6) 전수검사 DB에 들어갈 데이타프레임 작성

In [9]:
df3 = df3[['Mapping Model.Suffix', 'S/N', 'DOA Number', 'Repair Report']]

In [10]:
df3

,Mapping Model.Suffix,S/N,DOA Number,Repair Report
0,14T90R-G.AA56K,96451303QCHE570977,N5V23522H-391L,Replace KB
1,14T90R-G.AA56K,96451303QCLH571080,N5V23522H-391L,Repair MB
2,14T90R-G.AP56ML,24264304QCBD566277,N5V23522H-391L,Replace KB
3,14T90R-G.AP56ML,24264304QCPY566256,N5V23522H-391L,Replace LCD
4,14T90R-G.AP56ML,24264304QCXM566226,N5V23522H-391L,Replace T/P
5,15U40R-G.ARTWK,97487303QCYQ570786,N5V23522H-391L,Replace B cover
6,15U40R-G.ARTWK,97487303QCZP570597,N5V23522H-391L,Repair MB
7,15U50R-S.AP70ML,24290304QCQX567292,N5V23522H-391L,Replace D cover
8,15U50R-S.AP70ML,24290304QCWC567305,N5V23522H-391L,Replace D cover
9,16T90R-G.AA5CK,08361304QCGW556111,N5V23522H-391L,Repair adapter


In [11]:
df5 = df5[['S/N', '판정', '불량 내용']]
df5 = df5.rename(columns={'판정':'Judge', '불량 내용':'Defect Reason_Korean'})

In [12]:
df5

,S/N,Judge,Defect Reason_Korean
0,96451303QCHE570977,OK,NaN
1,96451303QCLH571080,OK,NaN
2,24264304QCBD566277,OK,NaN
3,24264304QCPY566256,OK,NaN
4,24264304QCXM566226,OK,NaN
5,97487303QCYQ570786,OK,NaN
6,97487303QCZP570597,OK,NaN
7,24290304QCQX567292,OK,NaN
8,24290304QCWC567305,OK,NaN
9,08361304QCGW556111,OK,NaN


In [13]:
df3 = df3.merge(df5)
df3

,Mapping Model.Suffix,S/N,DOA Number,Repair Report,Judge,Defect Reason_Korean
0,14T90R-G.AA56K,96451303QCHE570977,N5V23522H-391L,Replace KB,OK,NaN
1,14T90R-G.AA56K,96451303QCLH571080,N5V23522H-391L,Repair MB,OK,NaN
2,14T90R-G.AP56ML,24264304QCBD566277,N5V23522H-391L,Replace KB,OK,NaN
3,14T90R-G.AP56ML,24264304QCPY566256,N5V23522H-391L,Replace LCD,OK,NaN
4,14T90R-G.AP56ML,24264304QCXM566226,N5V23522H-391L,Replace T/P,OK,NaN
5,15U40R-G.ARTWK,97487303QCYQ570786,N5V23522H-391L,Replace B cover,OK,NaN
6,15U40R-G.ARTWK,97487303QCZP570597,N5V23522H-391L,Repair MB,OK,NaN
7,15U50R-S.AP70ML,24290304QCQX567292,N5V23522H-391L,Replace D cover,OK,NaN
8,15U50R-S.AP70ML,24290304QCWC567305,N5V23522H-391L,Replace D cover,OK,NaN
9,16T90R-G.AA5CK,08361304QCGW556111,N5V23522H-391L,Repair adapter,OK,NaN


In [14]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df1 = pickle.load(f)

In [15]:
df5 = pd.DataFrame(columns=df1.columns)
df5

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category


In [16]:
df5 = pd.concat([df5, df3])[df5.columns]
df5

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,NaN,14T90R-G.AA56K,96451303QCHE570977,OK,NaN,NaN,NaN,Replace KB,N5V23522H-391L,NaN,NaN,NaN
1,NaN,14T90R-G.AA56K,96451303QCLH571080,OK,NaN,NaN,NaN,Repair MB,N5V23522H-391L,NaN,NaN,NaN
2,NaN,14T90R-G.AP56ML,24264304QCBD566277,OK,NaN,NaN,NaN,Replace KB,N5V23522H-391L,NaN,NaN,NaN
3,NaN,14T90R-G.AP56ML,24264304QCPY566256,OK,NaN,NaN,NaN,Replace LCD,N5V23522H-391L,NaN,NaN,NaN
4,NaN,14T90R-G.AP56ML,24264304QCXM566226,OK,NaN,NaN,NaN,Replace T/P,N5V23522H-391L,NaN,NaN,NaN
5,NaN,15U40R-G.ARTWK,97487303QCYQ570786,OK,NaN,NaN,NaN,Replace B cover,N5V23522H-391L,NaN,NaN,NaN
6,NaN,15U40R-G.ARTWK,97487303QCZP570597,OK,NaN,NaN,NaN,Repair MB,N5V23522H-391L,NaN,NaN,NaN
7,NaN,15U50R-S.AP70ML,24290304QCQX567292,OK,NaN,NaN,NaN,Replace D cover,N5V23522H-391L,NaN,NaN,NaN
8,NaN,15U50R-S.AP70ML,24290304QCWC567305,OK,NaN,NaN,NaN,Replace D cover,N5V23522H-391L,NaN,NaN,NaN
9,NaN,16T90R-G.AA5CK,08361304QCGW556111,OK,NaN,NaN,NaN,Repair adapter,N5V23522H-391L,NaN,NaN,NaN


In [17]:
print(f"DOA 검사 불량율 : {df5.groupby('Judge')['S/N'].count()['NG'] / (df5.groupby('Judge')['S/N'].count()['OK'] + df5.groupby('Judge')['S/N'].count()['NG']) * 100:.2f} %")

KeyError: 'NG'

In [18]:
# 변수 설정
ins_date = datetime.datetime(2023,6,23) # 검사일 설정
rsps = 'LG'
Insp_reason = 'DOA units inspection'

In [19]:
df5['Inspection Date'] = ins_date
df5['Responsibility'] = rsps
df5['Inspection_Reason'] = Insp_reason
df5 # 전수검사 DB에 업데이트할 금번 DOA 전수검사 결과 리스트

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2023-06-23,14T90R-G.AA56K,96451303QCHE570977,OK,NaN,NaN,NaN,Replace KB,N5V23522H-391L,LG,DOA units inspection,NaN
1,2023-06-23,14T90R-G.AA56K,96451303QCLH571080,OK,NaN,NaN,NaN,Repair MB,N5V23522H-391L,LG,DOA units inspection,NaN
2,2023-06-23,14T90R-G.AP56ML,24264304QCBD566277,OK,NaN,NaN,NaN,Replace KB,N5V23522H-391L,LG,DOA units inspection,NaN
3,2023-06-23,14T90R-G.AP56ML,24264304QCPY566256,OK,NaN,NaN,NaN,Replace LCD,N5V23522H-391L,LG,DOA units inspection,NaN
4,2023-06-23,14T90R-G.AP56ML,24264304QCXM566226,OK,NaN,NaN,NaN,Replace T/P,N5V23522H-391L,LG,DOA units inspection,NaN
5,2023-06-23,15U40R-G.ARTWK,97487303QCYQ570786,OK,NaN,NaN,NaN,Replace B cover,N5V23522H-391L,LG,DOA units inspection,NaN
6,2023-06-23,15U40R-G.ARTWK,97487303QCZP570597,OK,NaN,NaN,NaN,Repair MB,N5V23522H-391L,LG,DOA units inspection,NaN
7,2023-06-23,15U50R-S.AP70ML,24290304QCQX567292,OK,NaN,NaN,NaN,Replace D cover,N5V23522H-391L,LG,DOA units inspection,NaN
8,2023-06-23,15U50R-S.AP70ML,24290304QCWC567305,OK,NaN,NaN,NaN,Replace D cover,N5V23522H-391L,LG,DOA units inspection,NaN
9,2023-06-23,16T90R-G.AA5CK,08361304QCGW556111,OK,NaN,NaN,NaN,Repair adapter,N5V23522H-391L,LG,DOA units inspection,NaN


In [20]:
df5['Defect Reason_Korean'] = df5['Defect Reason_Korean'].str.replace('\n', ',') # 불량 증상 문자열 중 '/n' 이 있으면 ',' 로 변경함

In [21]:
df5.loc[df5['Defect Reason_Korean'].notnull(), ['Defect Reason_Korean', 'Defect Reason_English']] # 불량 증상만 나온 열을 확인

,Defect Reason_Korean,Defect Reason_English


In [22]:
df5.loc[df5['Defect Reason_Korean'].notnull(), 'Defect Reason_English'] = df5[df5['Defect Reason_Korean'].notnull()]['Defect Reason_Korean'].apply(get_google_translate, args=['en']) # Google 번역
df5[df5['Defect Reason_Korean'].notnull()]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category


In [23]:
df5.loc[df5['Defect Reason_Korean'].notnull(), ('Defect Reason_Korean', 'Defect Reason_English')]

,Defect Reason_Korean,Defect Reason_English


In [25]:
df5.loc[df5['Defect Reason_Korean'].str.contains('작음', case=False, na=False), ('Defect Reason_Korean', 'Defect Reason_English')]

,Defect Reason_Korean,Defect Reason_English


In [80]:
df5.loc[df5['Defect Reason_Korean'].str.contains('작음', case=False, na=False), 'Defect Reason_English'] = 'Speaker-R sound weak'

In [26]:
df5[df5['Defect Reason_Korean'].notnull()]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
5,2023-06-13,14TD90R-G.AX56K,96484303QCTB568873,NG,NaN,System Fan 지속 구동,System Fan continuous operation,Replace C cover,N5V23424H-391L,LG,DOA units inspection,NaN
6,2023-06-13,14TD90R-G.AX56K,96484303QCTB571513,NG,NaN,System Fan 지속 구동,System Fan continuous operation,Replace KB,N5V23424H-391L,LG,DOA units inspection,NaN
8,2023-06-13,15U50Q-G.AP50ML,303QCSF559342,NG,NaN,"C/D-Part 파지시 소음, 박스불량","Noise when holding C/D-Part, defective box",Replace T/Pad,N5V23424H-391L,LG,DOA units inspection,NaN
9,2023-06-13,15U50Q-G.AR50ML,303QCLH560424,NG,NaN,박스불량,bad box,Replace KB,N5V23424H-391L,LG,DOA units inspection,NaN
10,2023-06-13,15UD40R-G.AX56K,303QCZP563517,NG,NaN,박스불량,bad box,Replace C cover,N5V23424H-391L,LG,DOA units inspection,NaN


In [27]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df = pickle.load(f)

In [28]:
df.shape

(111879, 12)

In [29]:
df5.shape

(25, 12)

In [30]:
df = pd.concat([df, df5]).reset_index(drop=True)
df

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
111899,2023-06-23,16T90R-G.AA5CK,08361304QCCV556179,OK,NaT,NaN,NaN,Replace B cover,N5V23522H-391L,LG,DOA units inspection,NaN
111900,2023-06-23,16T90R-G.AA5CK,08361304QCGW556159,OK,NaT,NaN,NaN,Replace KB,N5V23522H-391L,LG,DOA units inspection,NaN
111901,2023-06-23,16T90R-G.AA5CK,08361304QCJZ556196,OK,NaT,NaN,NaN,Replace B cover,N5V23522H-391L,LG,DOA units inspection,NaN
111902,2023-06-23,16T90R-G.AA5CK,08361304QCQX556132,OK,NaT,NaN,NaN,Replace C cover,N5V23522H-391L,LG,DOA units inspection,NaN


In [31]:
df.shape

(111904, 12)

In [32]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [33]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

### SD에 RMA Number가 없으면 기입

In [109]:
rma_num = 'N5V21927H-391'
df1.loc['889110028321', 'Inspection'] = rma_num

In [111]:
df1.shape

(1781, 38)

In [112]:
with open('D:/Data/SD_raw_data.bin', 'wb') as f:
    pickle.dump(df1, f)

## 해당 DOA 넘버에 해당하는 S/N 리스트 추출

In [72]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)
rma_num = 'N5V22316H-391L'
df1 = df1[df1['RMA Number'] == rma_num] # ODM 업체의 RMA 넘버에 해당하는 S/N 만 추출
df1.head(3)

,S/N,Vendor,Series,Mapping Model.Suffix,Price,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,RMA Number,HBL,OBD,Remark
1017,201QCAS589256,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,C-Part Scratch & Dent,C-Part Scratch & Dent,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN
1018,201QCAS589904,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,T/P 우측 상단 들뜸\nAccessory Partition 누락,T/P upper right is lifted_x000D_\nAccessory Pa...,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN
1019,201QCAS594944,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,A-Part 얼룩,A-Part Stain,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN


In [73]:
df1.shape

(261, 13)

In [299]:
# 업체(Pegatron, Quanta의 엑셀 수리 내역을 복사하여 데이타프레임으로 만듬
df2 = pd.read_clipboard()
df2.head(3)

,S/N,QTA PN,Vendor,Model,Price,FOC Number,Grouping Number,RMA Number,Repair Report,Cost($)
0,112QCNL583575,1NLH0000318,Quanta,14T90P-G.AA7BK,845.13,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace D cover,3
1,201QCLH580008,1NLH0000318,Quanta,14T90P-G.AA7BK,845.13,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace D cover,3
2,112QCBD592269,1NLH0000315,Quanta,14T90P-G.AA7DK,846.53,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace LCD,12


In [300]:
# S/N 자리수 check 하여 이상한 것 있으면 공백 제거함
df2.insert(1, 'S/N count', df2['S/N'].str.len())
df2[df2['S/N count']!=13]['S/N'].tolist()

[]

In [301]:
df2['S/N'] = df2['S/N'].str.strip() # 이상한 것이 발견되면 공백 제거 실시
df2['S/N count'] = df2['S/N'].str.len() # 공백 제거 후, 다시 자리수 check
df2[df2['S/N count']!=13]['S/N'].tolist() # 결과 확인

[]

In [302]:
# DOA return 한 S/N에 해당하는 업체의 각 S/N별 수리 내역을 Merge하여 새로운 df로 만듬
df3 = df1[['Mapping Model.Suffix', 'S/N', 'Defective_Symptom(KOR)', 'Defective_Symptom(ENG)']].merge(df2[['S/N', 'Repair Report']], how='outer')

In [303]:
df3['RMA Number'] = rma_num

In [304]:
df3[df3['Repair Report'].notnull()].shape # 만들어진 df 내용 확인]

(54, 6)

In [305]:
df3 = df3[df3['Repair Report'].notnull()]

### 특정 DOA number의 return한 S/N 개수에서 지금까지 입고되어 전수검사 완료된 S/N 제외해봄

In [85]:
# DOA Return DB에서 특정 DOA number로 Return 한 S/N 개수 확인
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)
df1 = df1[df1['RMA Number'] == rma_num]
df1.shape

(261, 13)

In [83]:
# 전수검사 DB에서 특정 DOA 로 들어온 전체 S/N 개수 확인
rma_num = 'N5V22316H-391L'

with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df = pickle.load(f)
df = df[df['RMA Number'] == rma_num]
df.shape

(119, 11)

In [88]:
print(f'아직 검사 안한 S/N 개수 : {df1.shape[0] - df.shape[0]}' )

아직 검사 안한 S/N 개수 : 142


In [308]:
blank_list = []
for col in df3.columns:
    if col in df.columns:
        blank_list.append(col)
blank_list

['Mapping Model.Suffix', 'S/N', 'Repair Report', 'RMA Number']

In [309]:
df = pd.concat([df, df3[blank_list]])

In [310]:
df = df.reset_index(drop=True) # 인덱스 재부여

In [311]:
df.shape

(83495, 11)

In [ ]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [312]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

### 2) 해당 수리 LOT에 입고검사 결과 붙이기

In [315]:
inspection_date = datetime.date(2022,6,18)

In [324]:
cond1 = df['RMA Number'] == rma_num

In [318]:
df[cond].shape

(119, 11)

In [319]:
# 입고 검사 결과 엑셀파일을 불러와서 불량율 맞는지 확인해보기
df_new = pd.read_clipboard()
df_new['판정'].value_counts(normalize=True)

OK    0.87037
NG    0.12963
Name: 판정, dtype: float64

In [320]:
df_new['S/N'].str.len().value_counts() # S/N 자리수가 13자리 맞는지 확인

13    54
Name: S/N, dtype: int64

In [321]:
df_new.columns

Index(['모델명', 'S/N', '생산지', '차용일', '불량 증상', '수리 내용', '판정', '불량 내용', '반납일'], dtype='object')

In [322]:
for sn, model, judge, defect in df_new[['S/N', '모델명', '판정', '불량 내용']].values:
    cond1 = (df['RMA Number'] == rma_num)
    cond2 = (df['S/N'] == sn)
    cond3 = (df['Mapping Model.Suffix'] == model)
    df.loc[cond1&cond2&cond3, ['Inspection Date', 'Judge', 'Defect Reason_Korean', 'Responsibility', 'Inspection_Reason']] = [
        inspection_date, judge, defect, 'LG', 'DOA units inspection']

In [323]:
cond4 = df['Defect Reason_Korean'].notnull()

In [325]:
df[cond1&cond4]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
83454,2022-06-18,14T90P-G.AAF6K,112QCZP588845,NG,NaT,LCD Open/Close Noise,NaN,Repair MB,N5V22316H-391L,LG,DOA units inspection
83456,2022-06-18,14T90P-G.AR5BK,201QCMR555915,NG,NaT,D-Part Dent,NaN,Replace C cover,N5V22316H-391L,LG,DOA units inspection
83460,2022-06-18,14T90P-G.AR5GK,112QCWC587345,NG,NaT,"D-Part 돌출, Box 불량",NaN,Replace T/P,N5V22316H-391L,LG,DOA units inspection
83461,2022-06-18,14T90P-G.AR5GK,112QCYQ583994,NG,NaT,D-Part 소음,NaN,Replace B cover,N5V22316H-391L,LG,DOA units inspection
83470,2022-06-18,14T90P-G.AR5GK,202QCRN561094,NG,NaT,B-Part Scratch/Dent,NaN,Replace LCD,N5V22316H-391L,LG,DOA units inspection
83479,2022-06-18,14TD90P-G.AX7GK,201QCJZ579084,NG,NaT,D-Part 돌출,NaN,Repair MB,N5V22316H-391L,LG,DOA units inspection
83489,2022-06-18,14TD90P-G.AX7GK,201QCYQ580146,NG,NaT,D-Part 돌출,NaN,Replace C cover,N5V22316H-391L,LG,DOA units inspection


In [326]:
df.loc[cond1&cond4, 'Defect Reason_English'] = df.loc[cond1&cond4, 'Defect Reason_Korean'].apply(get_translate, args=['en'])

In [327]:
df.loc[cond1&cond4]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
83454,2022-06-18,14T90P-G.AAF6K,112QCZP588845,NG,NaT,LCD Open/Close Noise,LCD Open/Close Noise,Repair MB,N5V22316H-391L,LG,DOA units inspection
83456,2022-06-18,14T90P-G.AR5BK,201QCMR555915,NG,NaT,D-Part Dent,D-Part Dent,Replace C cover,N5V22316H-391L,LG,DOA units inspection
83460,2022-06-18,14T90P-G.AR5GK,112QCWC587345,NG,NaT,"D-Part 돌출, Box 불량","D-part protrusion, box defective",Replace T/P,N5V22316H-391L,LG,DOA units inspection
83461,2022-06-18,14T90P-G.AR5GK,112QCYQ583994,NG,NaT,D-Part 소음,D-Part Noise,Replace B cover,N5V22316H-391L,LG,DOA units inspection
83470,2022-06-18,14T90P-G.AR5GK,202QCRN561094,NG,NaT,B-Part Scratch/Dent,B-Part Scratch/Dent,Replace LCD,N5V22316H-391L,LG,DOA units inspection
83479,2022-06-18,14TD90P-G.AX7GK,201QCJZ579084,NG,NaT,D-Part 돌출,D-Part protrusion,Repair MB,N5V22316H-391L,LG,DOA units inspection
83489,2022-06-18,14TD90P-G.AX7GK,201QCYQ580146,NG,NaT,D-Part 돌출,D-Part protrusion,Replace C cover,N5V22316H-391L,LG,DOA units inspection


In [ ]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [328]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)